<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=26686d36df1892817399d0f2fed89f1d59c98466dab8fe7a75c377e00cb1eefc
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-09 12:44:35
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: 13.30 K (0.1%)
Current PnL: -29.26 L (-18.9%)
CY Booked + Current PnL: -14.97 L (-9.67%)
-------------------
Total profit:  1.18 L
Total loss:  -30.44 L
-------------------
Total Booked + Current PnL: 12.01 L (9.39%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.28%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.89 L (67.54%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.73,-17.05,20.66,0.09,17271.0,-17179.0,83598.0,94.02,38.0,M-SC,2.21,253.0,-0.99,0.62,9.83,OX40N,NTT,DURABLES
43,ITC,452.00,0.27,-1.44,12.05,10.44,23770.0,-2875.0,197263.0,-41.15,47.0,X-LC,1.64,5.0,-0.12,1.47,4.05,X40,NTT,FMCG
39,INDIGOPNTS,1408.00,0.47,-14.09,16.42,0.02,24624.0,-24593.0,149966.0,128.79,48.0,M-SC,16.67,234.0,-1.00,1.12,30.44,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.87,-3.93,27.05,22.05,25461.0,-3855.0,94125.0,-17.34,52.0,H-SC,6.98,164.0,-0.15,0.70,36.82,XR,ATH,FINANCE
35,ICICIGI,2252.93,-0.35,4.26,15.28,20.19,27771.0,7430.0,181750.0,-18.38,42.0,X-MC,5.99,68.0,0.27,1.36,20.02,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
10,BANDHANBNK,400.0,3.85,-32.32,184.31,92.42,347470.0,-90034.0,188525.0,32.90,34.0,H-SC,7.86,170.0,-0.26,1.41,8.95,XY24,NTT,BANKS
80,VALIANTORG,838.0,3.82,-43.75,209.62,74.15,192894.0,-71584.0,92021.0,-265.32,47.0,H-SC,21.17,149.0,-0.37,0.69,17.96,XR,NTT,CHEMICALS
56,RAJESHEXPO,518.0,3.42,-63.81,176.53,0.07,87629.0,-87537.0,49640.0,1719.44,52.0,L-SC,2.36,268.0,-1.00,0.37,23.46,OX40N,NTT,JEWELLERY
79,VAIBHAVGBL,521.0,2.77,-27.55,121.90,60.77,161422.0,-50353.0,132422.0,52.62,43.0,H-SC,5.27,159.0,-0.31,0.99,19.26,XR,NTT,JEWELLERY
44,JCHAC,2282.0,2.38,-37.41,59.76,-0.01,47802.0,-47815.0,79990.0,16269.57,15.0,M-SC,3.29,233.0,-1.00,0.60,2.38,OX40N,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3460.25,-4.04,-1.79,23.14,20.95,57219.0,-4497.0,247271.0,-2.74,50.0,X-LC,19.04,31.0,-0.08,1.84,33.21,X40,ATH,PAINTS
6,ASIANTILES,137.00,-2.24,-21.26,130.72,81.67,96562.0,-19941.0,73869.0,6633.33,28.0,L-SC,14.76,271.0,-0.21,0.55,43.05,XR,NTT,CERAMICS
18,COFFEEDAY,80.00,-2.23,-47.24,156.41,35.27,93696.0,-53645.0,59904.0,-59.76,20.0,L-SC,12.39,270.0,-0.57,0.45,46.62,XR,NTT,HOTELS
49,LAOPALA,464.00,-1.88,-39.24,128.92,39.09,102694.0,-51448.0,79657.0,74.23,25.0,H-SC,7.37,155.0,-0.50,0.59,2.59,AR,NTT,CERAMICS
28,GREENPANEL,537.00,-1.59,-38.97,134.54,43.14,150324.0,-71346.0,111732.0,150.00,23.0,M-SC,8.64,240.0,-0.47,0.83,5.10,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.97,-8.28,126.69,107.91,171559.0,-12228.0,135416.0,-25.42,30.0,M-SC,11.88,216.0,-0.07,1.01,0.97,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.47,-14.09,16.42,0.02,24624.0,-24593.0,149966.0,128.79,48.0,M-SC,16.67,234.0,-1.00,1.12,30.44,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.73,-17.05,20.66,0.09,17271.0,-17179.0,83598.0,94.02,38.0,M-SC,2.21,253.0,-0.99,0.62,9.83,OX40N,NTT,DURABLES
66,SIS,528.00,-0.31,-25.41,62.56,21.26,51815.0,-28208.0,82824.0,1952.33,43.0,H-SC,4.63,166.0,-0.54,0.62,11.90,OX40N,NTT,MISC
47,KANSAINER,340.00,0.63,-25.49,52.30,13.47,105084.0,-68742.0,200925.0,-69.28,28.0,H-SC,3.19,158.0,-0.65,1.50,2.16,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-1.2,5.92,62.63,72.27,112411.0,10036.0,179484.0,-9.26,66.0,M-LC,2.23,99.0,0.09,1.34,12.46,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.20,5.92,62.63,72.27,112411.0,10036.0,179484.0,-9.26,66.0,M-LC,2.23,99.0,0.09,1.34,12.46,XR,NTT,IT
38,INDIAMART,4810.62,0.23,-2.08,111.11,106.72,134189.0,-2565.0,120771.0,-51.58,36.0,H-SC,1.21,139.0,-0.02,0.90,20.81,AR,ATH,MISC
25,FINCABLES,1641.55,0.97,-8.28,126.69,107.91,171559.0,-12228.0,135416.0,-25.42,30.0,M-SC,11.88,216.0,-0.07,1.01,0.97,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-0.49,-2.98,39.79,35.63,80905.0,-6243.0,203331.0,-16.74,35.0,H-MC,3.16,119.0,-0.08,1.52,13.20,AR,ATH,PHARMA
37,IEX,219.00,-0.58,-4.25,55.22,48.63,105724.0,-8495.0,191459.0,-36.24,47.0,H-SC,14.09,136.0,-0.08,1.43,6.81,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,1.75,-22.06,90.47,48.45,141383.0,-44230.0,156276.0,-27.60,20.0,X-MC,11.18,64.0,-0.31,1.17,1.75,X40N,ATH,FINANCE
22,DIXON,18931.72,-0.14,-13.02,39.78,21.58,48490.0,-18243.0,121896.0,-56.18,24.0,X-MC,5.45,56.0,-0.38,0.91,7.92,X40N,ATH,IT
75,TMPV,600.00,-0.27,-17.69,72.74,42.18,165494.0,-48896.0,227514.0,-26.74,27.0,X-LC,5.07,3.0,-0.30,1.70,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.00,0.25,-11.19,29.74,15.23,52916.0,-22417.0,177930.0,-24.06,28.0,X-MC,7.90,66.0,-0.42,1.33,17.45,X40N,NTT,REALTY
11,BATAINDIA,2096.00,1.27,-38.04,118.22,35.20,94247.0,-48948.0,79722.0,4.55,29.0,X-SC,15.27,92.0,-0.52,0.59,1.27,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-0.74,-21.27,94.97,53.50,225607.0,-64195.0,237556.0,-61.84,33.0,X-MC,1.26,58.0,-0.28,1.77,5.51,XY24,NTT,FMCG
43,ITC,452.0,0.27,-1.44,12.05,10.44,23770.0,-2875.0,197263.0,-41.15,47.0,X-LC,1.64,5.0,-0.12,1.47,4.05,X40,NTT,FMCG
3,ACC,3906.0,-0.65,-25.01,118.90,64.15,212165.0,-59511.0,178440.0,-56.31,35.0,X-SC,1.79,82.0,-0.28,1.33,0.10,XY24,BTT,CEMENT
15,CAMS,4762.0,-0.56,-80.37,535.61,24.79,264843.0,-202409.0,49447.0,-81.32,40.0,X-SC,1.84,86.0,-0.76,0.37,20.88,X40N,NTT,MISC
36,ICICIPRULI,790.0,1.63,3.83,26.14,30.97,47968.0,6768.0,183506.0,-20.57,56.0,X-MC,2.08,75.0,0.14,1.37,16.87,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.08,-0.09,-9.12,38.72,26.07,57615.0,-14940.0,148800.0,-33.02,30.0,X-MC,11.48,55.0,-0.26,1.11,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.27,-17.69,72.74,42.18,165494.0,-48896.0,227514.0,-26.74,27.0,X-LC,5.07,3.0,-0.30,1.70,0.00,XY24,NTT,AUTO
54,PGHH,17907.65,-0.99,-5.89,42.12,33.76,79607.0,-11820.0,189000.0,-33.58,34.0,X-MC,5.09,57.0,-0.15,1.41,0.10,X40,ATH,FMCG
3,ACC,3906.00,-0.65,-25.01,118.90,64.15,212165.0,-59511.0,178440.0,-56.31,35.0,X-SC,1.79,82.0,-0.28,1.33,0.10,XY24,BTT,CEMENT
31,HAVELLS,2069.16,0.53,-9.42,46.26,32.48,137432.0,-30904.0,297087.0,-14.05,39.0,X-MC,5.19,67.0,-0.22,2.22,0.93,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.13,-30.95,106.87,42.84,47970.0,-20120.0,44886.0,-54.82,37.0,X-SC,37.06,83.0,-0.42,0.33,1.55,XY24,NTT,MISC
15,CAMS,4762.00,-0.56,-80.37,535.61,24.79,264843.0,-202409.0,49447.0,-81.32,40.0,X-SC,1.84,86.0,-0.76,0.37,20.88,X40N,NTT,MISC
59,RELAXO,1176.00,1.17,-47.87,192.17,52.31,145415.0,-69490.0,75670.0,-44.54,37.0,X-SC,6.11,91.0,-0.48,0.56,1.29,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,1.27,-38.04,118.22,35.20,94247.0,-48948.0,79722.0,4.55,29.0,X-SC,15.27,92.0,-0.52,0.59,1.27,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.14,-13.02,39.78,21.58,48490.0,-18243.0,121896.0,-56.18,24.0,X-MC,5.45,56.0,-0.38,0.91,7.92,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-1.13,-13.05,37.19,19.28,111868.0,-45156.0,300800.0,-24.27,61.0,X-LC,5.77,1.0,-0.40,2.24,11.20,X40,ATH,IT
41,INFY,2275.00,-0.88,8.70,42.48,54.87,144473.0,27209.0,340097.0,-15.29,63.0,X-LC,3.38,2.0,0.19,2.54,17.70,X40,BTT,IT
75,TMPV,600.00,-0.27,-17.69,72.74,42.18,165494.0,-48896.0,227514.0,-26.74,27.0,X-LC,5.07,3.0,-0.30,1.70,0.00,XY24,NTT,AUTO
81,VBL,671.64,0.11,-5.00,42.81,35.67,128452.0,-15791.0,300051.0,-16.24,52.0,X-LC,4.82,4.0,-0.12,2.24,8.16,X40N,ATH,FMCG
43,ITC,452.00,0.27,-1.44,12.05,10.44,23770.0,-2875.0,197263.0,-41.15,47.0,X-LC,1.64,5.0,-0.12,1.47,4.05,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,0.87,-3.93,27.05,22.05,25461.0,-3855.0,94125.0,-17.34,52.0,H-SC,6.98,164.0,-0.15,0.70,36.82,XR,ATH,FINANCE
13,BSOFT,831.70,0.84,-21.73,92.63,50.79,101982.0,-30557.0,110096.0,0.50,70.0,H-SC,5.85,151.0,-0.30,0.82,28.19,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.47,-14.09,16.42,0.02,24624.0,-24593.0,149966.0,128.79,48.0,M-SC,16.67,234.0,-1.00,1.12,30.44,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-4.04,-1.79,23.14,20.95,57219.0,-4497.0,247271.0,-2.74,50.0,X-LC,19.04,31.0,-0.08,1.84,33.21,X40,ATH,PAINTS
68,STARHEALTH,761.00,0.97,-9.79,63.23,47.26,160359.0,-27510.0,253613.0,23.11,38.0,H-SC,13.23,171.0,-0.17,1.89,35.88,XY24,NTT,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.84,-21.73,92.63,50.79,101982.0,-30557.0,110096.0,0.50,70.0,H-SC,5.85,151.0,-0.30,0.82,28.19,XR,ATH,IT
32,HCLTECH,1922.01,-1.06,8.42,15.04,24.73,39450.0,20376.0,262300.0,13.18,66.0,X-LC,5.79,7.0,0.52,1.96,24.94,X40,ATH,IT
84,WIPRO,420.00,-1.20,5.92,62.63,72.27,112411.0,10036.0,179484.0,-9.26,66.0,M-LC,2.23,99.0,0.09,1.34,12.46,XR,NTT,IT
5,ASIANPAINT,3460.25,-4.04,-1.79,23.14,20.95,57219.0,-4497.0,247271.0,-2.74,50.0,X-LC,19.04,31.0,-0.08,1.84,33.21,X40,ATH,PAINTS
73,TCS,4389.96,-1.13,-13.05,37.19,19.28,111868.0,-45156.0,300800.0,-24.27,61.0,X-LC,5.77,1.0,-0.40,2.24,11.20,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.80
1,25,45.77
2,50,76.73


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.90
MC    30.35
LC    24.77
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.27
X40      22.52
X40N     10.89
XR       10.05
XY25      9.42
AR        9.34
OX40N     7.76
SR        1.02
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.50
X-MC    23.62
X-LC    21.20
M-SC    12.08
X-SC     6.95
H-MC     4.96
M-MC     1.41
L-SC     1.37
M-LC     1.34
H-LC     1.18
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.46,-7.47,42.23
IT,12.86,-16.95,80.17
FINANCE,9.52,-21.53,72.88
ELECTRICAL,6.24,-12.38,53.54
MISC,6.10,-58.57,123.53
PAINTS,6.09,-13.04,29.83
INSURANCE,4.62,-2.15,38.15
PHARMA,4.10,-1.68,34.22
AUTO,3.43,-25.00,78.60


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3243968.0,21
AR,1320846.0,10
XR,1316540.0,13
X40,1018765.0,14
X40N,990186.0,9
OX40N,753496.0,10
XY25,378470.0,6
SR,283707.0,2
MH,83360.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3696396.0,25
M-SC,1438410.0,15
X-MC,1317682.0,16
X-SC,985883.0,8
X-LC,920965.0,11
H-MC,413728.0,3
L-SC,277887.0,3
M-LC,112411.0,1
H-LC,75852.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1297303.0      6
           AR         899912.0      5
M-SC       XY24       818049.0      6
H-SC       XR         783574.0      7
X-SC       X40N       507226.0      3
X-MC       X40        457516.0      7
           XY24       394794.0      3
X-LC       X40        376780.0      5
H-SC       OX40N      348540.0      4
M-SC       OX40N      317327.0      5
X-SC       XY24       294188.0      3
H-SC       SR         283707.0      2
X-MC       X40N       279458.0      4
X-LC       XY24       243708.0      2
H-MC       AR         217802.0      2
X-LC       X40N       203502.0      2
H-MC       XY24       195926.0      1
L-SC       XR         190258.0      2
X-MC       XY25       185914.0      2
X-SC       X40        184469.0      2
M-SC       XR         175754.0      2
           AR         127280.0      2
M-LC       XR         112411.0      1
X-LC       XY25        96975.0      2
L-SC       OX40N       87629.0      1
H-SC       MH          83360.0      1
H-LC       AR          75852.0      1
M-MC       XY25        56593.0      1
L-MC       XR          54543.0      1
L-LC       XY25        38988.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
